<a href="https://colab.research.google.com/github/Mohammad-Rahmdel/Happy_Face_Recognition/blob/master/Happy_Face_Recognition_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! git clone https://github.com/Mohammad-Rahmdel/Happy_Face_Recognition

Cloning into 'Happy_Face_Recognition'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [0]:
import keras.backend as K
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt


def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

def load_dataset():
    train_dataset = h5py.File('Happy_Face_Recognition/datasets/train_happy.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('Happy_Face_Recognition/datasets/test_happy.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes




In [24]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model


import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.models import Sequential
from keras import regularizers



X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

X_train = X_train_orig/255.
X_test = X_test_orig/255.

Y_train = Y_train_orig.T
Y_test = Y_test_orig.T


def HappyModel(input_shape):
    """
    Implementation of the HappyModel.
    
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
   
    X_input = Input(shape= (input_shape,input_shape,3))
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='HappyModel')


    return model
  
  
  def HappyModel2(input_shape):

    weight_decay = 1e-4;
    baseMapNum = 32
    model = Sequential()
    model.add(Conv2D(filters=baseMapNum, kernel_size=(3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), 
    input_shape=(input_shape,input_shape,3)))
    model.add(Activation('relu'))

    model.add(BatchNormalization())
    model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))

    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))

    model.add(BatchNormalization())
    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))

    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))

    model.add(BatchNormalization())
    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))

    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model



"""
You have now built a function to describe your model. To train and test this model, there are four steps in Keras:
    1. Create the model by calling the function above
    2. Compile the model by calling model.compile(optimizer = "...", loss = "...", metrics = ["accuracy"])
    3. Train the model on train data by calling model.fit(x = ..., y = ..., epochs = ..., batch_size = ...)
    4. Test the model on test data by calling model.evaluate(x = ..., y = ...)
"""

happyModel = HappyModel(X_train.shape[1])

happyModel.compile(optimizer = "Adam", loss = "binary_crossentropy", metrics = ["accuracy"])
happyModel.fit(x = X_train, y = Y_train, epochs = 20, batch_size = 32, shuffle=True)
preds = happyModel.evaluate(X_train, Y_train)
print ("Test Accuracy = " + str(preds[1]))
preds = happyModel.evaluate(X_test, Y_test)
print ("Test Accuracy = " + str(preds[1]))

# happyModel.summary()


""" Result : 
HappyModel2
optimizer = "Adam", loss = "binary_crossentropy"
epochs = 15, batch_size = 32, shuffle=True
Test Accuracy = 0.9749999992052714
Test Accuracy = 0.9266666642824809

HappyModel
optimizer = "Adam", loss = "binary_crossentropy"
epochs = 15, batch_size = 32, shuffle=True
Test Accuracy = 0.9933333333333333
Test Accuracy = 0.9599999976158142
"""




Epoch 1/20
600/600 [==============================] - 4s 6ms/step - loss: 1.8385 - acc: 0.6367
Epoch 2/20
600/600 [==============================] - 1s 2ms/step - loss: 0.4372 - acc: 0.8550
Epoch 3/20
600/600 [==============================] - 1s 2ms/step - loss: 0.3029 - acc: 0.8917
Epoch 4/20
600/600 [==============================] - 1s 2ms/step - loss: 0.2550 - acc: 0.9250
Epoch 5/20
600/600 [==============================] - 1s 2ms/step - loss: 0.2274 - acc: 0.9367
Epoch 6/20
600/600 [==============================] - 1s 2ms/step - loss: 0.2638 - acc: 0.9217
Epoch 7/20
600/600 [==============================] - 1s 2ms/step - loss: 0.2202 - acc: 0.9350
Epoch 8/20
600/600 [==============================] - 1s 2ms/step - loss: 0.2029 - acc: 0.9383
Epoch 9/20
600/600 [==============================] - 1s 2ms/step - loss: 0.1554 - acc: 0.9533
Epoch 10/20
600/600 [==============================] - 1s 2ms/step - loss: 0.2122 - acc: 0.9567
Epoch 11/20
600/600 [============================